In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import gc

In [ ]:
articles = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/articles.csv",nrows=5000)
print(articles.columns.tolist())
customers = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/customers.csv",nrows=5000)
print(customers.columns.tolist())
transactions = pd.read_csv("../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv",nrows=500)
print(transactions.columns.tolist())

In [ ]:
#merge
trans_arti = transactions.merge(articles,how='inner',on="article_id")
trans_arti.columns.tolist()

In [ ]:
trans_arti.head(4)
print("The shape of the dataset",trans_arti.shape)

In [ ]:
gc.collect()
#need to reduce few unimportance Features /columns which can be further used for feature eng. but as here cant due to lack of kaggle RAM
trans_arti.head(3)
drop_list = ["index_code","index_name","index_group_no","section_no","detail_desc","prod_name","detail_desc"
             ]
trans_arti.drop(drop_list,inplace=True,axis=1)
trans_arti.head(3)
print("The shape of the dataset", trans_arti.shape)


In [ ]:
#merge2
df = trans_arti.merge(customers, how="inner", on="customer_id")
df.head(3)
print("The shape of the dataset", df.shape)

In [ ]:
#drop the unimportant features
drop_col1 =["postal_code"]
df.drop(drop_col1,inplace=True,axis=1)
print("The shape of the dataset", df.shape)

In [ ]:
#checking for missing values
df.isna().sum()

In [ ]:
#df[["Active","FN"]].dtypes
#wrong format 
df["Active"] = df["Active"].astype(object)
df["Active"].dtype
df["FN"] = df["FN"].astype(object)
#df["Active"].fillna(df["Active"].mode()[0],inplace=True)


In [ ]:
#Impute Method 1 with Xgboost
def impute_missing_wxgboost(df,conservative = False, n_feats = 10,            
           fix_string_nans = True, verbose = True,                
           multiprocessing_load = 3, fill_nans_in_pure_text = True,                    
           drop_empty_cols = True, drop_nan_cols_with_constant = False,                        
           ):
    
    from verstack import NaNImputer
    #verstack 2.0.1
    imputer = NaNImputer(conservative = conservative, n_feats = n_feats,            
           fix_string_nans = fix_string_nans, verbose = verbose,                
           multiprocessing_load = multiprocessing_load, fill_nans_in_pure_text = fill_nans_in_pure_text,                    
           drop_empty_cols = drop_empty_cols, drop_nan_cols_with_constant = drop_nan_cols_with_constant,                      
           )
    
    df_imputed = imputer.impute(df)
    
    return df_imputed

df1 = impute_missing_wxgboost(df)
df1.dtypes

In [ ]:
#Custom Impute Missing Values

#function to check impute missing values 
#1 checks for datatype
#2 if character impute mode 
#3 if numberical  check again skewness
#4 if skewed impute with median else mean
from scipy.stats import skew

def impute_values(df,skewness_threshold =0.25):
        for i in df.columns:
            #temp = df.dtypes
            if df[i].dtype == object:
                print("Character Variable Detected")
                print("Missing Value found",df[i].isna().sum())
                print("Ratio",df[i].isna().sum()/df.shape[0]*100)
                df[i].fillna(df[i].mode()[0],inplace=True)
                print("Missing Values After imputation",df[i].isna().sum())
            elif df[i].dtype == int or float:
                s_v = skew(df[i])
                print("Numeric Variable Detected")
                if (s_v >=skewness_threshold or s_v <= skewness_threshold):
                    print("Missing Value found",df[i].isna().sum())
                    print("Skewness detected")
                    df[i].fillna(df[i].median(),inplace=True)
                    print("Missing Values After imputation",df[i].isna().sum())
                else:
                    df[i].fillna(df[i].mean(skipna=True),inplace=True)
                    print("Missing Values After imputation",df[i].isna().sum())

            else:
                pass


df = impute_values(df)
            

    

In [ ]:
df.isna().sum()

In [ ]:
df.head(4)
#df.describe()

In [ ]:
#data is cleaned 

to be continued..